Intro to Langgraph
We build a simple research assistant where the llm will classifiy the given question as a research question or general question.
Based on the classification llm routes the question to specific nodes to handle research quesiton and general quesiton. 
Install required libraries - langchian azureopenai langgraph grandalf

In [ ]:
!pip install langgraph grandalf

Initialise your llm 

In [1]:
from langchain_community.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

import os
os.environ["OPENAI_API_BASE"] = "https://api.nlp.dev.uptimize.merckgroup.com"#"https://nlp.dev.uptimize.merckgroup.com"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "cbfe29d4-971a-4215-9030-6b674bb090d4"#get_nlp_api_key() # never store your API key in your code

# The extra headers won't be necessary in the future
headers = {
    "x-api-key": os.environ["OPENAI_API_KEY"],
}


llm = AzureChatOpenAI(
    deployment_name="gpt-4-32k-0613",
    model_name="gpt-4-32k-0613",
    max_tokens=1000
)

#response_chat =llm([HumanMessage(content="write a cool poem about air and water")])
#print(response_chat)


C:\Users\M319170\AppData\Local\miniconda3\envs\new_test\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
C:\Users\M319170\AppData\Local\miniconda3\envs\new_test\Lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://api.nlp.dev.uptimize.merckgroup.com to https://api.nlp.dev.uptimize.merckgroup.com/openai.
  warnings.warn(
C:\Users\M319170\AppData\Local\miniconda3\envs\new_test\Lib\

Initialise Graph state with questionm classification and response - this helps in deciding the action to be taken by llm

In [2]:
from typing import Dict, TypedDict, Optional
from langgraph.graph import StateGraph,END

class GraphState(TypedDict):
    question: Optional[str] = None
    classification: Optional[str] = None
    response: Optional[str] = None

workflow = StateGraph(GraphState)

Define a function where llm classifies the question as Research question or an General question.This is basically our Supervisor LLM that decides what to do. 

In [3]:
def classify(question):
    result=llm.invoke("""Classisfy the given question as an research question or an general question. If its a research question output it as 'Research' else 'General'.
    DO NOT provide any commetns, or appolocgies. Your output shoud contain only a single word. Here is the quesiton"""+question)
    return result.content
#classify("where can i find India")    

Function to handle classify question

In [5]:
def classify_input_node(state):
    question = state.get('question', '').strip()
    classification = classify(question)  # Assume a function that classifies the input
    return {"classification": classification}

In [6]:
#Function to handle research question
#this is a basic function. You can add your own functions to handle different questions and inputs
def handle_research_node(state):
    return {"response": f"Search result for '{question}'"}


#Function to handle general question
def handle_search_node(state):
    question = state.get('question', '').strip()
    search_result = f"Search result for '{question}'"
    return {"response": search_result}

here we define nodes along with its respective functions. Nodes can be either function or any runnables such as Langchain LCEL

In [7]:
workflow.add_node("classify_input", classify_input_node)
workflow.add_node("handle_research", handle_research_node)
workflow.add_node("handle_search", handle_search_node)

here we use a decision function and create an conditional edge. this helps the supervisor to route the tasks to specifc nodes.

In [8]:
def decide_next_node(state):
    return "handle_research" if state.get('classification') == "Research" else "handle_search"

workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "handle_research": "handle_research",
        "handle_search": "handle_search"
    }
)

Here we set an starting point of our graph using set entry point and add corresponding edges to it.

In [9]:
workflow.set_entry_point("classify_input")
workflow.add_edge('handle_research', END)
workflow.add_edge('handle_search', END)

now we complie the graph

In [10]:
app = workflow.compile()

In [11]:
inputs = {"question": "Hello, how are you?"}
result = app.invoke(inputs)
print(result)

{'question': 'Hello, how are you?', 'classification': 'General', 'response': "Search result for 'Hello, how are you?'"}


below is the simple visualization for your graph 

In [12]:
app.get_graph().print_ascii()

                  +-----------+                 
                  | __start__ |                 
                  +-----------+                 
                        *                       
                        *                       
                        *                       
               +----------------+               
               | classify_input |               
               +----------------+               
                        *                       
                        *                       
                        *                       
      +---------------------------------+       
      | classify_input_decide_next_node |       
      +---------------------------------+       
                ***           ***               
              **                 **             
            **                     **           
+-----------------+           +---------------+ 
| handle_research |           | handle_search | 
+-----------------+ 